#Fake News Detection

## Identifying Textual Relationships with Deep Neural Nets

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
# Set current working directory so that all relative path references work

root = "/content/drive/My Drive/PGP-AIML/Fake News"

%cd $root

/content/drive/My Drive/PGP-AIML/Fake News


In [0]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import shutil

import pandas as pd
import numpy as np

import itertools

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


### Loading the Glove Embeddings

In [0]:
# from zipfile import ZipFile
# with ZipFile(join(root, 'glove.6B.zip'), 'r') as z:
#   z.extractall()

### Load the dataset

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
train_bodies = pd.read_csv("train_bodies.csv")
train_stances = pd.read_csv("train_stances.csv")

In [0]:
print("TRAIN BODIES\n" + '-' * 12)
print("Shape   : {}".format(train_bodies.shape))
print("Columns : [ {} ]".format(", ".join(train_bodies.columns)))
print("")
print("TRAIN STANCES\n" + '-' * 13)
print("Shape   : {}".format(train_stances.shape))
print("Columns : [ {} ]".format(", ".join(train_stances.columns)))


TRAIN BODIES
------------
Shape   : (1683, 2)
Columns : [ Body ID, articleBody ]

TRAIN STANCES
-------------
Shape   : (49972, 3)
Columns : [ Headline, Body ID, Stance ]


In [0]:
train_df = train_bodies.merge(train_stances, on='Body ID')

print("MERGED TRAIN DATA\n" + '-' * 17)
print("Shape   : {}".format(train_df.shape))
print("Columns : [ {} ]".format(", ".join(train_df.columns)))

MERGED TRAIN DATA
-----------------
Shape   : (49972, 4)
Columns : [ Body ID, articleBody, Headline, Stance ]


In [0]:
train_df.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Tokenizing the text and loading the pre-trained Glove word embeddings for each token 

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence

In [0]:
tmp = Tokenizer(num_words=4, oov_token='OUT_OF_VOCAB')

tmp_texts = ["This is some text to test this tokenizer thing! I heard the tokenizer thing is pretty cool."]

tmp.fit_on_texts(tmp_texts)

tmp.word_index


{'OUT_OF_VOCAB': 1,
 'cool': 14,
 'heard': 11,
 'i': 10,
 'is': 3,
 'pretty': 13,
 'some': 6,
 'test': 9,
 'text': 7,
 'the': 12,
 'thing': 5,
 'this': 2,
 'to': 8,
 'tokenizer': 4}

In [0]:
tmp_words = text_to_word_sequence(tmp_texts[0])

tmp_words


['this',
 'is',
 'some',
 'text',
 'to',
 'test',
 'this',
 'tokenizer',
 'thing',
 'i',
 'heard',
 'the',
 'tokenizer',
 'thing',
 'is',
 'pretty',
 'cool']

In [0]:
list(itertools.chain.from_iterable(tmp.texts_to_sequences(tmp_words)))

[2, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1]

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
texts = []

texts = train_df['articleBody'].tolist() + train_df['Headline'].tolist()
print(len(texts))

tokenizer.fit_on_texts(texts)

99944


In [0]:
len(tokenizer.word_index)

27873

In [0]:
tokenizer.document_count

99944

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.


In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
article_texts = train_df['articleBody'].tolist()

In [0]:
article_sents = []

for article in article_texts:
  article_sents.append(nltk.sent_tokenize(article))

In [0]:
article_texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [0]:
article_sents[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

In [0]:
# Slicing using an index greater than the length of a list just returns the entire list.
# Multiplying a list by a negative value returns an empty list.
def trim_or_pad(lst, n):
  return lst[:n] + [0]*(n-len(lst))

In [0]:
article_encodings = []

for i in range(len(article_sents)):
  sentence_encodings = []
  n_sents = 0
  for sentence in article_sents[i]:
    # Limit number of sentences to MAX_SENTS
    if(n_sents < MAX_SENTS):
      print("Working on article no. {}, sentence no. {}".format(i, n_sents), end="\r")
      n_sents += 1

      words = text_to_word_sequence(sentence)
      word_encodings = list(itertools.chain.from_iterable(tokenizer.texts_to_sequences(words)))
      word_encodings = trim_or_pad(word_encodings, MAX_SENT_LENGTH)
      
      sentence_encodings.append(word_encodings)
    else:
      break
    
  # Handle the case where an article has number of sentences less than MAX_SENTS 
  while(n_sents < MAX_SENTS):
    print("Working on article no. {}, sentence no. {}".format(i, n_sents), end="\r")
    n_sents += 1

    empty_encodings = trim_or_pad([], MAX_SENT_LENGTH)
    sentence_encodings.append(empty_encodings)

  article_encodings.append(sentence_encodings)

In [0]:
article_encodings = np.array(article_encodings)
article_encodings.shape

(49972, 20, 20)

In [0]:
article_encodings[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
         1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
          555,   235],
       [   89,  1067,  4115,  2349,    12,     3,  1092,  3306,    19,
            1,    89,     2,  1793,     1,   521,  2009,    15,     9,
            3,  3111],
       [  181,  3640,   972,   200,  2556,    44,  6775,  1722,  1252,
            5, 13317, 17936,     1,   778,    31,   740,  3990,    67,
           85,     0],
       [ 2349,    12,  1557,    38,  1094,   351,   775,     2,   367,
          260,  1770,     5,  4450,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159,  1252,     6,     3,  5270,     4,  1217,  1252,
           12,  3363],
       [  

In [0]:
headline_texts = train_df['Headline'].tolist()

In [0]:
headline_sents = []

for headline in headline_texts:
  headline_sents.append(nltk.sent_tokenize(headline))

In [0]:
headline_encodings = []

for i in range(len(headline_sents)):
  sentence_encodings = []
  n_sents = 0
  for sentence in headline_sents[i]:
    # Limit number of sentences to MAX_SENTS_HEADING
    if(n_sents < MAX_SENTS_HEADING):
      print("Working on headline no. {}, sentence no. {}".format(i, n_sents), end="\r")
      n_sents += 1

      words = text_to_word_sequence(sentence)
      word_encodings = list(itertools.chain.from_iterable(tokenizer.texts_to_sequences(words)))
      word_encodings = trim_or_pad(word_encodings, MAX_SENT_LENGTH)
      
      sentence_encodings.append(word_encodings)
    else:
      break
    
  # Handle the case where an headline has number of sentences less than MAX_SENTS_HEADING 
  while(n_sents < MAX_SENTS_HEADING):
    print("Working on headline no. {}, sentence no. {}".format(i, n_sents), end="\r")
    n_sents += 1

    empty_encodings = trim_or_pad([], MAX_SENT_LENGTH)
    sentence_encodings.append(empty_encodings)

  headline_encodings.append(sentence_encodings)

In [0]:
headline_encodings = np.array(headline_encodings)
headline_encodings.shape

(49972, 1, 20)

In [0]:
headline_encodings[0, :, :]

array([[  718,   206,   343,  7134,   193,    34,  1338, 11554,    21,
          233,   686,     0,     0,     0,     0,     0,     0,     0,
            0,     0]])

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = pd.get_dummies(train_df['Stance'])
labels.columns

Index(['agree', 'disagree', 'discuss', 'unrelated'], dtype='object')

In [0]:
labels = labels.values
labels.shape

(49972, 4)

In [0]:
print("Shape of articles tensor : {}".format(article_encodings.shape))
print("Shape of headlines tensor : {}".format(headline_encodings.shape))
print("Shape of labels tensor : {}".format(labels.shape))

Shape of articles tensor : (49972, 20, 20)
Shape of headlines tensor : (49972, 1, 20)
Shape of labels tensor : (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(article_encodings.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
article_encodings = article_encodings[indices]

headline_encodings = headline_encodings[indices]

labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.

In [0]:
X_train, X_val, y_train, y_val = train_test_split(article_encodings, labels, test_size=0.2, random_state=10)

In [0]:
X_headline_train, X_headline_val, y_headline_train, y_headline_val = train_test_split(headline_encodings, labels, test_size=0.2, random_state=10)

In [0]:
X_train = np.reshape(X_train, (X_train.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
print(X_train.shape)

X_val = np.reshape(X_val, (X_val.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
print(X_val.shape)

print(y_train.shape)
print(y_val.shape)

(39977, 400)
(9995, 400)
(39977, 4)
(9995, 4)


In [0]:
X_headline_train = np.reshape(X_headline_train, (X_headline_train.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
print(X_headline_train.shape)

X_headline_val = np.reshape(X_headline_val, (X_headline_val.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
print(X_headline_val.shape)

print(y_headline_train.shape)
print(y_headline_val.shape)

(39977, 20)
(9995, 20)
(39977, 4)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((MAX_NB_WORDS, 100))


for word, i in tokenizer.word_index.items():
  if i >= MAX_NB_WORDS: 
    continue

  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


## Try a bidirectional LSTM model and report the accuracy score

### Model

In [0]:
input_ = Input(shape=(MAX_SENTS * MAX_SENT_LENGTH, ))
x = Embedding(MAX_NB_WORDS, 100, weights=[embedding_matrix])(input_) # embed size is 100 given that we've used glove.6B.100d.txt
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(4, activation="softmax")(x)
model = Model(inputs=input_, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 400)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 400, 100)          2000000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 400, 200)          160800    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 404       
Total para

### Compile and fit the model

In [0]:
model.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_val, y_val))

Train on 39977 samples, validate on 9995 samples
Epoch 1/5
39977/39977 [==============================] - 1364s 34ms/step - loss: 0.3256 - acc: 0.8698 - val_loss: 0.2927 - val_acc: 0.8800
Epoch 2/5
39977/39977 [==============================] - 1363s 34ms/step - loss: 0.2769 - acc: 0.8893 - val_loss: 0.2634 - val_acc: 0.8992
Epoch 3/5
39977/39977 [==============================] - 1353s 34ms/step - loss: 0.2565 - acc: 0.8982 - val_loss: 0.2501 - val_acc: 0.9040
Epoch 4/5
39977/39977 [==============================] - 1352s 34ms/step - loss: 0.2452 - acc: 0.9031 - val_loss: 0.2431 - val_acc: 0.9048
Epoch 5/5
39977/39977 [==============================] - 1357s 34ms/step - loss: 0.2362 - acc: 0.9066 - val_loss: 0.2386 - val_acc: 0.9084


In [0]:
input_ = Input(shape=(MAX_SENTS_HEADING * MAX_SENT_LENGTH, ))
x = Embedding(MAX_NB_WORDS, 100, weights=[embedding_matrix])(input_) # embed size is 100 given that we've used glove.6B.100d.txt
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(4, activation="softmax")(x)
model = Model(inputs=input_, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 100)           2000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20, 200)           160800    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total para

In [0]:
model.fit(X_headline_train, y_headline_train, batch_size=64, epochs=5, validation_data=(X_headline_val, y_headline_val))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 39977 samples, validate on 9995 samples
Epoch 1/5
39977/39977 [==============================] - 92s 2ms/step - loss: 0.3464 - acc: 0.8649 - val_loss: 0.3348 - val_acc: 0.8642
Epoch 2/5
39977/39977 [==============================] - 90s 2ms/step - loss: 0.3297 - acc: 0.8658 - val_loss: 0.3279 - val_acc: 0.8642
Epoch 3/5
39977/39977 [==============================] - 97s 2ms/step - loss: 0.3207 - acc: 0.8657 - val_loss: 0.3260 - val_acc: 0.8642
Epoch 4/5
39977/39977 [==============================] - 97s 2ms/step - loss: 0.3148 - acc: 0.8656 - val_loss: 0.3263 - val_acc: 0.8632
Epoch 5/5
39977/39977 [==============================] - 96s 2ms/step - loss: 0.3111 - acc: 0.8657 - val_loss: 0.3269 - val_acc: 0.8642
